# Big Data & BI – Intro Notebook

**Topic:** Data, Data Types, Databases, Warehouses & Lakes


## Learning Objectives
By the end, you can:
- Explain **structured/semi-structured/unstructured** data (with examples).
- Contrast a **Data Warehouse** with a **Data Lake**.
- Run simple **SQL** on a toy database.
- Git commit check

In [1]:
import pandas as pd
import numpy as np
import json, sqlite3, os, io, textwrap
from pprint import pprint
# comment

In [2]:
# Pandas display options
pd.set_option("display.max_rows", 8)
pd.set_option("display.max_colwidth", 80)



## 1) What is Data? The 3 Vs
**Data** = recorded observations/measurements. In Big Data, remember the **3 Vs**:
- **Volume** (how much)
- **Velocity** (how fast)
- **Variety** (how many kinds)

Let's create a toy multi-format dataset (orders, sensor stream, comments).


In [3]:
# Create three tiny datasets: structured, semi-structured, unstructured

# Structured data: Orders DataFrame
orders = pd.DataFrame({
    "order_id":[101,102,103,104,105],
    "customer":["Ava","Ben","Ava","Luca","Mina"],
    "item":["Pizza","Salad","Pasta","Pasta","Pizza"],
    "price":[12.5,8.0,10.0,10.0,12.5],
    "ts": pd.date_range("2025-01-01", periods=5, freq="h")
})
orders


,order_id,customer,item,price,ts
0,101,Ava,Pizza,12.5,2025-01-01 00:00:00
1,102,Ben,Salad,8.0,2025-01-01 01:00:00
2,103,Ava,Pasta,10.0,2025-01-01 02:00:00
3,104,Luca,Pasta,10.0,2025-01-01 03:00:00
4,105,Mina,Pizza,12.5,2025-01-01 04:00:00


In [4]:
# Semi-structured: JSON logs (nested-shallow because of meta field)
logs = [
    {"event":"click","user":"Ava","path":"/home","meta":{"device":"mobile","utm":"spring"}},
    {"event":"view","user":"Mina","path":"/menu","meta":{"device":"desktop"}},
    {"event":"purchase","user":"Ava","path":"/checkout","meta":{"device":"mobile","coupon":"WELCOME10"}}
]
pprint(logs)


[{'event': 'click',
  'meta': {'device': 'mobile', 'utm': 'spring'},
  'path': '/home',
  'user': 'Ava'},
 {'event': 'view',
  'meta': {'device': 'desktop'},
  'path': '/menu',
  'user': 'Mina'},
 {'event': 'purchase',
  'meta': {'coupon': 'WELCOME10', 'device': 'mobile'},
  'path': '/checkout',
  'user': 'Ava'}]


In [5]:
# Unstructured: free text comments
comments = [
    "Pizza was 🔥 and delivery was fast!",
    "Salad okay, dressing too sweet.",
    "Pasta great. Would reorder. Thanks :)"
]
for i, c in enumerate(comments, 1):
    print(f"{i}. {c}")


1. Pizza was 🔥 and delivery was fast!
2. Salad okay, dressing too sweet.
3. Pasta great. Would reorder. Thanks :)



**Excersice:** In your own words, map each object above to **Volume/Velocity/Variety**.  
*(No code needed—just discuss.)*



## 2) Data Types in Action
We'll parse the semi-structured JSON logs, and do a simple NLP analysis on unstructured text.


In [6]:
# Parse the JSON logs into a DataFrame (semi-structured -> structured view)
logs_df = pd.json_normalize(logs, sep="_")
logs_df


,event,user,path,meta_device,meta_utm,meta_coupon
0,click,Ava,/home,mobile,spring,NaN
1,view,Mina,/menu,desktop,NaN,NaN
2,purchase,Ava,/checkout,mobile,NaN,WELCOME10


In [9]:
# Quick text processing (unstructured): word counts ignoring tiny words
import re
def word_counts(lines):
    words = re.findall(r"[\w']+", " ".join(lines).lower())
    words = [w for w in words if len(w) > 3]
    return pd.Series(words).value_counts().head(10)

word_counts(comments)


delivery    2
pizza       1
fast        1
salad       1
           ..
sweet       1
pasta       1
great       1
would       1
Name: count, Length: 10, dtype: int64


**Excersice:** Add a new comment containing the word **'delivery'** twice and rerun the word count.  
**Hint:** `comments.append("...")`


In [8]:
# Goes here.
comments.append("The delivery was late wouldn't recommend")


## 3) Key Terms – Mini Glossary
Run the cell below and then call `define("schema")`, `define("oltp")`, etc.


In [10]:

GLOSSARY = {
    "schema": "The layout/structure of data (columns, types, constraints).",
    "database": "A system to store/retrieve data efficiently (e.g., PostgreSQL).",
    "sql": "Structured Query Language for relational databases.",
    "json": "Semi-structured text format with key–value pairs and nesting."
}

def define(term:str):
    key = term.strip().lower()
    print(f"{term} → {GLOSSARY.get(key, 'Unknown term. Try one of: ' + ', '.join(GLOSSARY))}")

print("Try: define('schema')")


Try: define('schema')


In [11]:
define("schema")

schema → The layout/structure of data (columns, types, constraints).


In [19]:
define("oltp")

oltp → Unknown term. Try one of: schema, database, sql, json



## 4) Data Lifecycle (Toy Example)
We'll simulate: **Collect → Store → Process → Analyze → Decide** using our orders.


In [12]:

# Collect: We already have 'orders'. Let's 'Store' by writing CSV (just as demo).
csv_path = "../data/orders.csv"
orders.to_csv(csv_path, index=False)

# Process: Clean simple things (e.g., ensure price is numeric & non-negative)
orders_clean = orders.assign(price = pd.to_numeric(orders["price"], errors="coerce")).query("price >= 0").copy()

# Analyze: basic KPIs
kpis = {
    "num_orders": len(orders_clean),
    "unique_customers": orders_clean["customer"].nunique(),
    "revenue": float(orders_clean["price"].sum()),
    "avg_ticket": float(orders_clean["price"].mean())
}
kpis


{'num_orders': 5, 'unique_customers': 4, 'revenue': 53.0, 'avg_ticket': 10.6}


**Decision:** Based on revenue and average ticket, we might test a **bundle** (e.g., *Pasta + Salad = €16*) to increase basket size.



## 5) Data Warehouse vs Data Lake – Tiny Demo
- **Warehouse idea (schema-on-write):** enforce a clean table with defined types.
- **Lake idea (schema-on-read):** drop raw JSON, interpret when you query.


In [13]:
conn = sqlite3.connect(":memory:")
orders.to_sql("orders", conn, index=False, if_exists="replace")

# Lake: store 'raw' as-line JSON (no enforced schema)
raw_path = "../data/raw_events.ndjson"
with open(raw_path, "w", encoding="utf-8") as f:
    for row in logs:
        f.write(json.dumps(row) + "\n")

# Warehouse: create a 'clean' table with explicit columns
conn.execute("DROP TABLE IF EXISTS dw_events")
conn.execute("""
CREATE TABLE dw_events(
    event TEXT,
    user TEXT,
    path TEXT,
    device TEXT,
    coupon TEXT
)
""")
# Transform raw -> structured and load 
for row in logs:
    conn.execute(
        "INSERT INTO dw_events VALUES (?,?,?,?,?)",
        (row.get("event"), row.get("user"), row.get("path"),
         row.get("meta",{}).get("device"), row.get("meta",{}).get("coupon"))
    )
conn.commit()

print("Wrote raw NDJSON to", raw_path)
pd.read_sql_query("SELECT * FROM dw_events", conn)


Wrote raw NDJSON to ../data/raw_events.ndjson


,event,user,path,device,coupon
0,click,Ava,/home,mobile,None
1,view,Mina,/menu,desktop,None
2,purchase,Ava,/checkout,mobile,WELCOME10



**Observation:** In the **lake** we kept everything raw; in the **warehouse** we defined columns upfront and loaded only the fields we care about.



## 6) Mini Project – From Raw to Insight
**Goal:** Given raw logs + orders, identify a customer segment and propose a data-driven action.

**Steps**
1. Create a feature: number of events per user (from `logs_df`).
2. Join with revenue per user (from `orders_clean`).
3. Label **high-engagement** users and propose a marketing action.


In [14]:
# 1) Events per user
ev = logs_df.groupby("user").size().rename("events")
# 2) Revenue per user
rev = orders_clean.groupby("customer")["price"].sum().rename("revenue")
# 3) Join
seg = pd.concat([ev, rev], axis=1)
seg.index.name = "user"
seg = seg.fillna(0).sort_values(["revenue","events"], ascending=False)
seg


,events,revenue
user,,
Ava,2.0,22.5
Mina,1.0,12.5
Luca,0.0,10.0
Ben,0.0,8.0



**Excercise** Define high-engagement users as those with `events >= 2` **or** `revenue >= 20`.  
Then, print a short recommendation for each high-engagement user.



## 7) Quick Quiz (self-check)
1. **Structured vs Semi-Structured:** Give one example of each from this notebook.  
3. **Warehouse vs Lake:** Which one uses **schema-on-read**?  

*(Answers at bottom)*



<details>
<summary>Quiz Answers</summary>

1. Structured: `orders` DataFrame; Semi-structured: `logs` JSON.  
3. Data Lake = schema-on-read.  
</details>



## 8) Wrap-Up 
- You practiced working with data in multiple formats.
- You contrasted warehouse vs lake.

